#### librerias

In [1]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

from sklearn.model_selection import train_test_split

## Parte 1

La primera parte del proyecto estara centrada en la prediccion categorica de canciones, se quiere predecir si una cancion con x caracteristicas puede ser de agrado para un agente Y. Esto tratara de imitar el modelo de seleccion de canciones nuevas que plataformas como Spotify, Apple music y demas productos de streaming ofrecen al usuario como nuevas alternativas musicales o descubrimientos musicales.

La informacion fue extraida de la API de Spotify para un agente X, con el cual trataremos de hacer un modelo que nos pueda aprender la relacion entre las caracteristicas de una cancion y el gusto musical de nuestro agente X, con el fin de poder en un futuro se pueda seleccionar un grupo de canciones, pasarlar por le modelo para obtener las que le gustarian a nuestro agente X y asi poder compartirle "nuevas canciones" que seran de su agrado. 

El modelo en si es basicamente funcionara como un sistema de recomendacion.

#### Dataset

In [2]:
raw_dataset1 = pd.read_csv("parte1/data.csv", index_col=0)

El dataset consiste en un set de canciones reproducidas sobre un agente X, por cada cancion contamos con 15 features que provee spotify y la variable respuesta que indica si la cancion tuvo fue de agrado para el agente o no.

**Variables:**
* acousticness: valor entre 0 y 1 que indica que tan acustica es una cancion.
* danceability: valor entre 0 y 1 que indica que tan indicada es la cancion para bailar, esto es medido a partir del tempo, ritmo, fuerza de los tonos.
* duration_ms: Durations de la cancion en milisegundos.
* energy: valor entre 0 y 1 que mide la velocidad, lo ruidoso y el alto tono de la cancion.
* instrumentalness: valor entre 0 y 1 que mide que tan instrumental es la cancion al verificar la ausencia de sonidos vocales.
* key: la llave de la cancion.
* liveness: valor entre 0 y 1 que detenta la presencia de audiencia en la cancion.
* loudness: valor entre -60 y 0 que mide los decibeles de volumen de la cancion.
* mode: entero entre 0 y 1 que indica la modalidad (mayor o menor) de una cancion basado en que tipo de escala es derivada la cancion.
* speechiness: valor entre 0 y 1 que detecta la presencia de palabras en la cancion.
* tempo: indica el tempo estimado de toda la cancion en tonos por minuto.
* time_signature: indica cuantos tonos hay en una barra.
* valence: valor entre 0 y 1 para describir la positividad de la cancion.
* song_title: nombre de la cancion
* artist: nombre del artista de la cancion

**Variable Respuesta:**
* target: Indica con 1 si tuvo like y 0 si no.

In [3]:
raw_dataset1.head(5)

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,target,song_title,artist
0,0.0102,0.833,204600,0.434,0.021900,2,0.1650,-8.795,1,0.4310,150.062,4.0,0.286,1,Mask Off,Future
1,0.1990,0.743,326933,0.359,0.006110,1,0.1370,-10.401,1,0.0794,160.083,4.0,0.588,1,Redbone,Childish Gambino
2,0.0344,0.838,185707,0.412,0.000234,2,0.1590,-7.148,1,0.2890,75.044,4.0,0.173,1,Xanny Family,Future
3,0.6040,0.494,199413,0.338,0.510000,5,0.0922,-15.236,1,0.0261,86.468,4.0,0.230,1,Master Of None,Beach House
4,0.1800,0.678,392893,0.561,0.512000,5,0.4390,-11.648,0,0.0694,174.004,4.0,0.904,1,Parallel Lines,Junior Boys


In [4]:
raw_dataset1.columns

Index(['acousticness', 'danceability', 'duration_ms', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
       'speechiness', 'tempo', 'time_signature', 'valence', 'target',
       'song_title', 'artist'],
      dtype='object')

Debido a que el dataset contiene variables identificadoras, estas seran eliminadas y ya que es un dataset que se obtuvo a traves del API de spotify las variables categoricas ya han sido convertidas a numericas, siendo estas representadas por numeros enteros. A continuacion se hara el split entre los conjuntos de entrenamiento y de prueba.

In [102]:
dataset1 = raw_dataset1[['acousticness', 'danceability', 'duration_ms', 'energy',
       'instrumentalness', 'liveness', 'loudness', 'mode',
       'speechiness', 'tempo', 'time_signature', 'valence', 'target']]

In [153]:
train, test_validation = train_test_split(dataset1, test_size=0.6, random_state=42)
test, validation = train_test_split(test_validation, test_size=0.5, random_state=42)

train_ds = torch.utils.data.TensorDataset(torch.tensor(train.drop('target', axis=1).values.astype(np.float32)), 
                                       torch.tensor(train['target'].values.astype(np.float32)))

test_ds = torch.utils.data.TensorDataset(torch.tensor(test.drop('target', axis=1).values.astype(np.float32)), 
                                       torch.tensor(test['target'].values.astype(np.float32)))

validation_ds = torch.utils.data.TensorDataset(torch.tensor(validation.drop('target', axis=1).values.astype(np.float32)), 
                                       torch.tensor(validation['target'].values.astype(np.float32)))
# dataset1_features = torch.tensor(dataset1.drop('target', axis=1).values.astype(np.float32))
# dataset1_target = torch.tensor(dataset1['target'].values.astype(np.float32))

# dataset1_tensor = torch.utils.data.TensorDataset(dataset1_features, dataset1_target)

# tp, te, vp = int(dataset1.shape[0] * 0.6), int(dataset1.shape[0] * 0.2), int(dataset1.shape[0] * 0.2) + 1
# train, test_validation = torch.utils.data.random_split(dataset1_tensor, [tp, te + vp], generator=torch.Generator().manual_seed(42))
# test, validation = torch.utils.data.random_split(test_validation, [te,vp], generator=torch.Generator().manual_seed(42))

#### Definicion arquitectura

In [213]:
class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        self.layer_1 = nn.Linear(12, 64) 
        self.layer_2 = nn.Linear(64, 64)
        self.layer_out = nn.Linear(64, 1) 
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)
        self.batchnorm1 = nn.BatchNorm1d(64)
        self.batchnorm2 = nn.BatchNorm1d(64)
        
        torch.nn.init.xavier_uniform_(self.layer_1.weight)
        torch.nn.init.zeros_(self.layer_1.bias)
        torch.nn.init.xavier_uniform_(self.layer_2.weight)
        torch.nn.init.zeros_(self.layer_2.bias)
        torch.nn.init.xavier_uniform_(self.layer_out.weight)
        torch.nn.init.zeros_(self.layer_out.bias)
        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.dropout(x)
        x = self.layer_out(x)
        
        return x
    
device = 'cpu'
model = NN()
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)        
epochs = 300
batch_size = 64
train_dataloader = DataLoader(train_ds, batch_size=batch_size)
test_dataloader = DataLoader(test_ds, batch_size=batch_size)

In [214]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = acc * 100
    
    return acc

In [216]:
def validation(dataloader, model):
    correct = 0
    with torch.no_grad():
        for batch, (X, y) in enumerate(dataloader):
            pred = model(X)
            pred = torch.round(torch.sigmoid(pred))
            correct += (pred == y.reshape((-1,1))).sum().float()
        return correct / len(dataloader.dataset) * 100

In [ ]:
losses = []
accs = []

In [217]:
for t in range(epochs):
    for batch, (X, y) in enumerate(train_dataloader):
        pred = model(X)
        loss = loss_fn(pred, y.reshape(-1, 1))
        acc = binary_acc(pred, y.reshape(-1, 1))

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if t % 50 == 0:
        print(f"Epoch {t+1}\n-------------------------------")
        validation_acc = validation(test_dataloader, model)
        losses.append(loss)
        accs = []
        print(f"loss: {loss:>7f} | accuracy: {acc:>2f} | validation_accuracy: {validation_acc: >2f}")
print("Done!")

Epoch 1
-------------------------------
loss: 0.683760 | accuracy: 44.736843 | validation_accuracy: 48.429752
Epoch 51
-------------------------------
loss: 0.611025 | accuracy: 68.421051 | validation_accuracy: 55.537189
Epoch 101
-------------------------------
loss: 0.626287 | accuracy: 63.157890 | validation_accuracy: 57.685955
Epoch 151
-------------------------------
loss: 0.623405 | accuracy: 65.789474 | validation_accuracy: 58.016525
Epoch 201
-------------------------------
loss: 0.581939 | accuracy: 73.684212 | validation_accuracy: 61.157024
Epoch 251
-------------------------------
loss: 0.516553 | accuracy: 76.315788 | validation_accuracy: 63.471073
Done!


#### Experimentacion

#### Conclusiones

## Parte 2

#### Dataset

#### Definicion arquitectura

#### Experimentacion

#### Conclusiones

## Parte 3

#### Dataset

#### Definicion arquitectura

#### Experimentacion

#### Conclusiones